## SERIAL DILUTION

### Methods:

1. Evenly spaced serial dilution with stock concentration, lowest concentration, number of columns, and final volume as inputs. 

2. Evenly spaced serial dilution with dilution factor (ex. 1/10), number of columns, and final volume as inputs

3. Custom dilution concentrations (not evenly spaced) with stock concentration, list of desired concentrations, and final volume as inputs

### Default Deck Locations:
1. Tip Box - 200uL **
2. Empty (HEAT NEST)
3. Stock plate (Corning 3383) -> place stock in Column 1
4. Serial Dilution plate (Corning 3383) -> empty to start
5. Diluent plate (12 channel reservoir or 96 deep well) -> place diluent in Column 1
6. Empty
7. Empty **
8. Empty 

** 50uL Tip Box may be placed at Position 1 or Position 7. This will be determined by volume calculations later on in the program. Please set up the deck as indicated by the cell later in this program. 

NOTE: This program works correctly as long as units are kept consistent. uL/uL or ng/uL serial dilutions will work. 


<font color='blue'>Which serial dilution method are you using? (1, 2, or 3) Enter it below.  </font>

In [14]:
method_number = 3

***

### Variables: 
<br>
<font color='blue'>Enter the following variables. These are required for all methods.</font>

In [15]:
output_hso_filename = "serial_dilution.hso"  # File name of generated .hso file

highest_concentration = 1/1000   # stock solution concentration (Enter as "None" or 0 if using method 2)

num_columns = 4              # Number of columns to use in dilution
desired_final_volume = 100   # Final Voume in wells after dilution is complete.

num_mixes = 5
do_blowoff = True    # yes = True, no = False 
stock_volume = 700   # volume of stock solution in one well (uL) - used to calculate mixing volume before transfer


<font color='blue'>Enter the variables for the method number you have chosen</font>
<br><br>
<font color='purple'>METHOD 1 VARIABLES:</font>

In [16]:
 lowest_concentration = 1/16000

<font color='purple'>METHOD 2 VARIABLES:</font>

In [17]:
dilution_factor = 1/10   # desired dilution factor/ratio (ex. 2-fold dilution = 1/2 = 0.5)

<font color='purple'>METHOD 3 VARIABLES:</font>

In [18]:
# enter a list of concentrations in uL (ex. [(1/2000), (1/4000), (1/16000)])
dilution_concentrations_list = [(1/2000), (1/4000), (1/40000), (1/160000)]  

***

### Code:
<br> 
<font color='purple'>IMPORT STATEMENTS</font>

In [19]:
import os
import sys
from liquidhandling import SoloSoft
from liquidhandling import GenericPlate96Well, NinetySixDeepWell, ZAgilentReservoir_1row

<font color='purple'>CALCULATIONS</font>
<br><br>
<font color='blue'>Calculations will be printed below this cell. Check that they match your expectations</font>

In [20]:
# Method 1 or 2
if method_number == 1 or method_number == 2:
    # calculate dilution factor and transfer volumes
    
    if method_number == 1:
        dilution_factor = (lowest_concentration/highest_concentration) ** (1/num_columns)
        
    serial_transfer_volume = int(dilution_factor * desired_final_volume)  
    diluent_transfer_volume = int(desired_final_volume - serial_transfer_volume)
    
    # calculate final volume in each well (TODO -> put calculation into a method?)
    final_volumes = [diluent_transfer_volume]*num_columns
    final_volumes[0] += serial_transfer_volume
    for i in range(num_columns-1):
        final_volumes[i] -= serial_transfer_volume
        final_volumes[i+1] += serial_transfer_volume
    
    # See calculation output below
    if method_number == 1:
        print("Method 1:" + "\n dilution factor = " + str(dilution_factor) 
              + "\n diluent transfer volume (uL) = " + str(diluent_transfer_volume) 
              + "\n serial transfer volume (uL) = " + str(serial_transfer_volume) 
              + "\n final volume in each well (uL) = " + str(final_volumes))
    elif method_number == 2:
        print("Method 2: \n diluent transfer volume (uL) = " + str(diluent_transfer_volume) 
              + "\n serial transfer volume (uL) = " + str(serial_transfer_volume)
              + "\n final volume in each well (uL) = " + str(final_volumes))
          
# Method 3 
elif method_number == 3:
    num_columns = len(dilution_concentrations_list)
    serial_transfer_volumes = [0]*num_columns
    diluent_transfer_volumes = [0]*num_columns
    for i in range(len(dilution_concentrations_list)):
        if i == 0: 
            current_dilution_factor = (dilution_concentrations_list[i]/highest_concentration) 
        else:
            current_dilution_factor = (dilution_concentrations_list[i]/dilution_concentrations_list[i-1])
        serial_transfer_volumes[i] = int(current_dilution_factor * desired_final_volume)
        diluent_transfer_volumes[i]  = int(desired_final_volume - serial_transfer_volumes[i])
 
    # calculate final volume in each well
    final_volumes = diluent_transfer_volumes.copy()
    for i in range(num_columns):
        final_volumes[i] += serial_transfer_volumes[i]
        if i < num_columns-1:
            final_volumes[i] -= serial_transfer_volumes[i+1]
          
    # See calculation output below
    print("Method 3: \n desired concentrations list (uL/uL) = " + str(dilution_concentrations_list) 
          + "\n diluent transfer volumes (uL) = " + str(diluent_transfer_volumes) 
          + "\n serial transfer volumes (uL) = " + str(serial_transfer_volumes)
          + "\n final volume in each well (uL) = " + str(final_volumes))
    

Method 3: 
 desired concentrations list (uL/uL) = [0.0005, 0.00025, 2.5e-05, 6.25e-06]
 diluent transfer volumes (uL) = [50, 50, 90, 75]
 serial transfer volumes (uL) = [50, 50, 10, 25]
 final volume in each well (uL) = [50, 90, 75, 100]


<font color='purple'>GENERATE DECK LAYOUT (and determine what tips are necessary)</font>

In [21]:
# Variables
plate_list = [
        "TipBox.200uL.Corning-4864.orangebox",
        "Empty",
        "Corning 3383",
        "Corning 3383",
        "12 Channel Reservoir",
        "Empty",
        "Empty",
        "Empty",
    ]

user_readable_plate_list = [
        "TipBox-200uL",
        "Empty - HEAT NEST",
        "Stock plate - Corning 3383",
        "Serial dilution plate - Corning 3383",
        "Diluent plate - 12 Channel Reservoir",
        "Empty",
        "Empty",
        "Empty",
    ]

tips_name_50uL = "TipBox.50uL.Axygen-EV-50-R-S.tealbox"
tips_location_50uL = None

diluent_50uL = False  
serial_50uL = False
only_50uL = False


# Method 1 and 2 -> only 50uL tips if all transfer volumes less than 20uL for now
if not method_number == 3:
    
    if serial_transfer_volume < 20:
        serial_50uL = True
    if diluent_transfer_volume < 20:
        diluent_50uL = True
    if serial_50uL == True and diluent_50uL == True:
        only_50uL = True
            
    print(serial_50uL)
    print(diluent_50uL)
    print("Only 50uL tips? -> " + str(only_50uL))

# Method 3
elif method_number == 3:
    
    serial_tip_locations = [[200, "Position1"]]*num_columns  # 200uL tips is the default
    diluent_tip_locations = [[200, "Position1"]]*num_columns
    
    if max(serial_transfer_volumes) <= 50 and max(diluent_transfer_volumes) <= 50:
        # no 200uL tips needed if max of all transfer volumes is 50 uL
        serial_tip_locations = [[50, "Position1"]]*num_columns  
        diluent_tip_locations = [[50, "Position1"]]*num_columns
        only_50uL = True
        
    else:
        for i in range(len(serial_transfer_volumes)):
            if serial_transfer_volumes[i] < 20: 
                serial_tip_locations[i] = [50, "Position7"]
                serial_50uL = True
            if diluent_transfer_volumes[i] < 20:
                diluent_tip_locations[i] = [50, "Position7"]
                diluent_50uL = True


# Determine deck position of 50 uL tips (if needed) -> do this for all methods
if only_50uL == True:
    plate_list[0] = tips_name_50uL
    user_readable_plate_list[0] = "TipBox-50uL"
    tips_location_50uL = "Position1"
elif (diluent_50uL == False and serial_50uL == True) or (diluent_50uL == True and serial_50uL == False):
    plate_list[6] = tips_name_50uL
    user_readable_plate_list[6] = "TipBox-50uL"
    tips_location_50uL = "Position7"

# for method 3 only... no need to switch from 50uL to 200uL tips if next trasfer volume <= 50uL
    # have to do this after the locaiton of the 50uL tip box has been established
if method_number == 3:  
    for i in range(1, num_columns):
        if serial_tip_locations[i][0] > serial_tip_locations[i-1][0] and serial_transfer_volumes[i] <= 50:
            serial_tip_locations[i] = [50, tips_location_50uL]
        if diluent_tip_locations[i][0] > diluent_tip_locations[i-1][0] and diluent_transfer_volumes[i] <= 50:
            diluent_tip_locations[i] = [50, tips_location_50uL]
    
    print("Serial tip locations: " + str(serial_tip_locations))
    print("Diluent tip locaitons: " + str(diluent_tip_locations))
    
    
print("\nLocation of 50uL tips: " + str(tips_location_50uL))




Serial tip locations: [[200, 'Position1'], [200, 'Position1'], [50, 'Position7'], [50, 'Position7']]
Diluent tip locaitons: [[200, 'Position1'], [200, 'Position1'], [200, 'Position1'], [200, 'Position1']]

Location of 50uL tips: Position7


***

<font color='blue'>----- **PLEASE SET UP THE DECK ACCORDINGLY** ------</font>

In [22]:
print("DECK LAYOUT:")
for i in range(len(user_readable_plate_list)):
    print(str(i+1) + " -> " + str(user_readable_plate_list[i]))
    
# TODO: add in some notes to the user about defalt positions and column numbers for each plate (in beginning?)

DECK LAYOUT:
1 -> TipBox-200uL
2 -> Empty - HEAT NEST
3 -> Stock plate - Corning 3383
4 -> Serial dilution plate - Corning 3383
5 -> Diluent plate - 12 Channel Reservoir
6 -> Empty
7 -> TipBox-50uL
8 -> Empty


***

<font color='purple'>CALCULATE MIXING VOLUMES</font>

In [23]:
# TODO: refactor this to prevent repeated code
# TODO: account for volume taken from stock solution well each time 

stock_mix_volumes = [0]*num_columns   
dilution_mix_volumes = [0]*num_columns 

# Methods 1 and 2
if not method_number == 3:
    if tips_location_50uL == None:
        stock_mix_volumes = [int(.6 * 200)]*num_columns if stock_volume > 200 else [int(.6 * stock_volume)]*num_columns
        for i in range(num_columns):
            dilution_mix_volumes[i] = int(final_volumes[i]*.6) if (final_volumes[i]*.6) < 200 else int(200*.6)
    
    elif diluent_50uL == True and serial_50uL == False:
        if not method_number == 3: 
            stock_mix_volumes = [int(.6*50)]*num_columns if stock_volume > 50 else [int(.6*stock_volume)]*num_columns
            for i in range(num_columns):
                dilution_mix_volumes[i] = int(final_volumes[i]*.6) if (final_volumes[i]*.6) < 200 else int(200*.6)
    
    elif diluent_50uL == False and serial_50uL == True:
        if not method_number == 3: 
            stock_mix_volumes = [int(.6*200)]*num_columns if stock_volume > 200 else [int(.6*stock_volume)]*num_columns
            for i in range(num_columns):
                dilution_mix_volumes[i] = int(final_volumes[i]*.6) if (final_volumes[i]*.6) < 50 else int(50*.6)

    elif only_50uL == True:
        if not method_number == 3:  
            stock_mix_volumes = [int(.6*50)]*num_columns if stock_volume > 50 else [int(.6*stock_volume)]*num_columns
            for i in range(num_columns):
                dilution_mix_volumes[i] = int(final_volumes[i]*.6) if (final_volumes[i]*.6) < 50 else int(50*.6)

# Method 3
elif method_number == 3:
    for i in range(num_columns):
        # stock mix volumes
        if stock_volume > diluent_tip_locations[i][0]:
            stock_mix_volumes[i] = int(diluent_tip_locations[i][0] * .6)
        else: 
            stock_mix_volumes[i] = int(stock_volume * .6)  # TODO: make this calculate the current stock volume!!!
        #dilution mix volumes
        if final_volumes[i] <= serial_tip_locations[i][0]:
            dilution_mix_volumes[i] = int(final_volumes[i]*.6)
        else:
            dilution_mix_volumes[i] = int(.6*serial_tip_locations[i][0]) 
            
print("Dilution mixing volumes: " + str(dilution_mix_volumes))
print("Stock mixing volumes: " + str(stock_mix_volumes))  
            
        

Dilution mixing volumes: [30, 54, 30, 30]
Stock mixing volumes: [120, 120, 120, 120]


<font color='purple'>CALCULATE BLOWOFF VOLUME(S) IF NECESSARY</font>

In [24]:
default_blowoff = 10

# for method 1 and 2
diluent_blowoff_volume = 0
serial_blowoff_volume = 0

# for method 3
diluent_blowoff_volumes = [0]*num_columns
serial_blowoff_volumes = [0]*num_columns

if do_blowoff == True:
    # Methods 1 and 2
    if not method_number == 3:
        if only_50uL == True: 
            diluent_blowoff_volume = min(default_blowoff, 50-diluent_transfer_volume) 
            serial_blowoff_volume = min(default_blowoff, 50-serial_transfer_volume)
        else:
            diluent_blowoff_volume = min(default_blowoff, 50-diluent_transfer_volume) if diluent_50uL == True else min(default_blowoff, 200-diluent_transfer_volume)
            serial_blowoff_volume = min(default_blowoff, 50-serial_transfer_volume) if serial_50uL == True else min(default_blowoff, 200-serial_transfer_volume)
        print(diluent_blowoff_volume)
        print(serial_blowoff_volume)
    
    # Method 3
    elif method_number == 3:
        for i in range(num_columns):
            diluent_blowoff_volumes[i] = min(default_blowoff, diluent_tip_locations[i][0]-diluent_transfer_volumes[i])
            serial_blowoff_volumes[i] = min(default_blowoff, serial_tip_locations[i][0]-serial_transfer_volumes[i])
                     
        print("Diluent blowoff volumes: " + str(diluent_blowoff_volumes))
        print("Serial blowoff volumes: " + str(serial_blowoff_volumes))
                
                
    


Diluent blowoff volumes: [10, 10, 10, 10]
Serial blowoff volumes: [10, 10, 10, 10]


<font color='purple'>GENERATE SOLOSOFT .HSO FILE</font>

In [25]:
# initialize soloSolft
# soloSoft = SoloSoft.SoloSoft(
#     filename=output_hso_filename,
#     plateList=plate_list,
# )

soloSoft = SoloSoft(
    filename=output_hso_filename,
    plateList=plate_list,
)

# METHODS 1 and 2
if method_number == 1 or method_number == 2:
    
    # determine the correct tip box locations
    diluent_tip_location = "Position1" if diluent_50uL == False else tips_location_50uL
    serial_tip_location = "Position1" if serial_50uL == False else tips_location_50uL
    
    # distribute diluent into all required wells 
    soloSoft.getTip(diluent_tip_location) 
    for i in range(1,num_columns+1):  # maybe add blowoff
        soloSoft.aspirate(  
            position="Position5", 
            aspirate_volumes=ZAgilentReservoir_1row().setColumn(1, diluent_transfer_volume),
            aspirate_shift=[0,0,4], # larger z-shift needed for 12 channel reservoir
            pre_aspirate=diluent_blowoff_volume,
        )
        soloSoft.dispense(
            position="Position4",
            dispense_volumes=GenericPlate96Well().setColumn(i, diluent_transfer_volume), 
            dispense_shift=[0,0,2], 
            blowoff=diluent_blowoff_volume,
        )
    
    # get the correct size tips for the serial dilution steps
    if not serial_tip_location == diluent_tip_location:
        soloSoft.getTip(serial_tip_location)
        
    # dilute into first column from stock solution
    soloSoft.aspirate(
        position="Position3", 
        aspirate_volumes=GenericPlate96Well().setColumn(1, serial_transfer_volume),  # TODO make sure the user places stock solution in this location
        aspirate_shift = [0,0,2], 
        mix_at_start=True,
        mix_cycles=num_mixes,
        mix_volume=stock_mix_volumes[0],
        dispense_height=2,
        pre_aspirate=serial_blowoff_volume,
    )
    soloSoft.dispense(
        position="Position4",
        dispense_volumes=GenericPlate96Well().setColumn(1, serial_transfer_volume), 
        dispense_shift=[0,0,2], 
        mix_at_finish=True, 
        mix_cycles=num_mixes, 
        mix_volume=dilution_mix_volumes[0],
        aspirate_height=2,
        blowoff=serial_blowoff_volume,
    )

    # serial dilute into the remaining columns
    for i in range(1,num_columns):  
        soloSoft.aspirate(
            position="Position4", 
            aspirate_volumes=GenericPlate96Well().setColumn(i, serial_transfer_volume),  # TODO make sure the user places stock solution in this location
            aspirate_shift = [0,0,2], 
            mix_at_start=True,
            mix_cycles=num_mixes,
            mix_volume=dilution_mix_volumes[i-1],
            dispense_height=2,
            pre_aspirate=serial_blowoff_volume,
        )
        soloSoft.dispense(
            position="Position4",
            dispense_volumes=GenericPlate96Well().setColumn(i+1, serial_transfer_volume), 
            dispense_shift=[0,0,2], 
            mix_at_finish=True, 
            mix_cycles=num_mixes, 
            mix_volume=dilution_mix_volumes[i],
            aspirate_height=2,
            blowoff=serial_blowoff_volume,
        )

    soloSoft.shuckTip()
    soloSoft.savePipeline()

# METHOD 3
elif method_number == 3: 
    # dispense predetermined differing amounts of diluent to each well
    soloSoft.getTip(diluent_tip_locations[0][1])  # the first diluent tip locaiton 
    for i in range(1,num_columns+1): # +1 means 1,2,...,num_columns
        soloSoft.aspirate(
            position="Position5", 
            aspirate_volumes=ZAgilentReservoir_1row().setColumn(1, diluent_transfer_volumes[i-1]),
            aspirate_shift=[0,0,4],
            pre_aspirate=diluent_blowoff_volumes[i-1],
        )
        soloSoft.dispense(
            position="Position4", 
            dispense_volumes=GenericPlate96Well().setColumn(i, diluent_transfer_volumes[i-1]), 
            dispense_shift=[0,0,2],
            blowoff=diluent_blowoff_volumes[i-1],
        )
        if not i > (num_columns-1):   # Change tip sizes if necessary
            if not diluent_tip_locations[i-1][0] == diluent_tip_locations[i][0]:
                soloSoft.getTip(diluent_tip_locations[i][1])
        
    # dispense the predetermined correct amount of stock solution into the first column
    if not diluent_tip_locations[-1][0] == serial_tip_locations[0][0]:  # switch to correct tip type of necessary
        soloSoft.getTip(serial_tip_locations[0][1])

    soloSoft.aspirate(
        position="Position3", 
        aspirate_volumes=GenericPlate96Well().setColumn(1, serial_transfer_volumes[0]),  # TODO make sure the user places stock solution in this location
        aspirate_shift = [0,0,2], 
        mix_at_start=True,
        mix_cycles=num_mixes,
        mix_volume=stock_mix_volumes[0],
        dispense_height=2,
        pre_aspirate=serial_blowoff_volumes[0],
    )
    soloSoft.dispense(
        position="Position4",
        dispense_volumes=GenericPlate96Well().setColumn(1, serial_transfer_volumes[0]), 
        dispense_shift=[0,0,2], 
        mix_at_finish=True, 
        mix_cycles=num_mixes, 
        mix_volume=dilution_mix_volumes[0],
        aspirate_height=2,
        blowoff=serial_blowoff_volumes[0],
    )

    # serial dilute into remaining columns 
    for i in range(1,num_columns): 
        # change tips if necessary
        if not serial_tip_locations[i][0] == serial_tip_locations[i-1][0]:
            soloSoft.getTip(serial_tip_locations[i][1])
        soloSoft.aspirate(
            position="Position4", 
            aspirate_volumes=GenericPlate96Well().setColumn(i, serial_transfer_volumes[i]),  # TODO make sure the user places stock solution in this location
            aspirate_shift = [0,0,2], 
            mix_at_start=True,
            mix_cycles=num_mixes,
            mix_volume=dilution_mix_volumes[i],
            dispense_height=2,
            pre_aspirate=serial_blowoff_volumes[i],
        )
        soloSoft.dispense(
            position="Position4",
            dispense_volumes=GenericPlate96Well().setColumn(i+1, serial_transfer_volumes[i]), 
            dispense_shift=[0,0,2], 
            mix_at_finish=True, 
            mix_cycles=num_mixes, 
            mix_volume=dilution_mix_volumes[i],
            aspirate_height=2,
            blowoff=serial_blowoff_volumes[i],
        )

    soloSoft.shuckTip()
    soloSoft.savePipeline()

In [26]:
# TODO: Run protocol in softlinx and auto start/remote start the program